## Screening features for high-dimensional datasets
## Demonstration

This notebook provides demonstration of how to use the unsupersived screening of features based on their variance. See HTML files for theoretical discussion on alternative implementations of the proposed procedure.

**Summary:**
1. [Libraries](#libraries)<a href='#libraries'></a>.
2. [Functions and classes](#functions_classes)<a href='#functions_classes'></a>.
3. [Unsupervised screening of features](#unsupervised_screening)<a href='#unsupervised_screening'></a>.
    * [Importing datasets](#import)<a href='#import'></a>.
    * [Features selection based on variance](#variance_based_selection)<a href='#variance_based_selection'></a>.
        * [Default implementation](#default)<a href='#default'></a>.
        * [Variance thresholding](#var_thres)<a href='#var_thres'></a>.
        * [Winsorize treatment](#winsorize)<a href='#winsorize'></a>.
        * [Dropping outliers](#drop_outliers)<a href='#drop_outliers'></a>.
        * [Handling collinearity](#collinearity)<a href='#collinearity'></a>.
    <br>
    <br>
    * [Features selection based on correlation](#corr_based_selection)<a href='#corr_based_selection'></a>.

<a id='libraries'></a>

## Libraries

In [1]:
import pandas as pd
import numpy as np
import json
import os

<a id='functions_classes'></a>

## Functions and classes

In [2]:
import screening_features
from screening_features import VarScreeningNumerical, CorrScreeningNumerical

<a id='unsupervised_screening'></a>

## Unsupervised screening of features

<a id='import'></a>

### Importing datasets

In [3]:
df_train = pd.read_csv('../Datasets/demo_dataset.csv', dtype={'order_id': str})

# Accessory variables:
drop_vars = drop_vars = ['y', 'id']

print('\033[1mShape of df_train:\033[0m ' + str(df_train.shape) + '.')
df_train.head()

Shape of df_train: (1282, 1652).


,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_1643,feat_1644,feat_1645,feat_1646,feat_1647,feat_1648,feat_1649,feat_1650,y,id
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11912002374
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12001000011
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12001000012
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12001000015
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12001000019


#### Scaling data

In [4]:
# Means for each feature:
means_dict = dict(zip(df_train.drop(drop_vars, axis=1).mean().index,
                      df_train.drop(drop_vars, axis=1).mean().values))

# Loop over features:
for f in means_dict.keys():
    df_train[f] = df_train[f].apply(lambda x: x/means_dict[f])

<a id='variance_based_selection'></a>

### Features selection based on variance

The screening of features based on variances involves two classes of selection: one for categorical variables, and the other for numerical (continuous) variables. The screening of categorical features considers one-hot encoding and a criterion for excluding dummy variables with very low variance (i.e., categories with rare or extremely common occurence). Since this procedure is rather straightforward, only the screening of numerical features is exemplified here.

Below, the collinearity filter will be applied to select numerical features together with the sorting of features by variance.

In [5]:
# Number of features to be selected:
new_p = 351

# Names of numerical features:
cont_vars = [c for c in df_train.columns if c not in drop_vars]

<a id='default'></a>

#### Default implementation

In [6]:
# Declaring object for screening of feature:
screening_cont = VarScreeningNumerical(features = cont_vars, na_features = [], stat='variance',
                                       select_k=True, k=new_p,
                                       thresholding=False, variance_threshold=0,
                                       winsorize=False, winsorize_param=0.025,
                                       drop_outliers=False, drop_outliers_param=0.01,
                                       collinearity=False, collinearity_param=0.9)

# Screening features:
screening_cont.select_feat(df_train.drop(drop_vars, axis=1))

# Dataframe with variance by feature:
variation_num_feat = screening_cont.var_continuous_feat

# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

/home/matheus_rosso/Arquivo/Materiais/Codes/high_dimensionality/Codes/screening_features.py:95: RuntimeWarning: Degrees of freedom <= 0 for slice.
  'variation': [np.nanvar(data[f]) for f in self.features]})


In [7]:
# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

print('updated_new_p: ' + str(screening_cont.k) + '.')
print('Number of selected features: ' + str(len(selected_feat)) + '.')
print('Number of features with zero variance: ' + str(len(zero_variance_feat)) + '.')

updated_new_p: 351.
Number of selected features: 351.
Number of features with zero variance: 272.


In [8]:
# Variance by numerical variable:
print('\033[1mShape of variation_num_feat:\033[0m ' + str(variation_num_feat.shape) + '.')
variation_num_feat.head(10)

Shape of variation_num_feat: (1378, 3).


variance,feature,variation,missing
636,feat_637,1281.000000,0
591,feat_592,1281.000000,0
909,feat_910,1281.000000,0
983,feat_984,1281.000000,0
1123,feat_1124,1281.000000,0
1121,feat_1122,1281.000000,0
661,feat_662,1281.000000,0
1533,feat_1534,1273.454465,0
1067,feat_1068,1027.765432,0
1065,feat_1066,898.180556,0


<a id='var_thres'></a>

#### Variance thresholding

In [9]:
# Declaring object for screening of feature:
screening_cont = VarScreeningNumerical(features = cont_vars, na_features = [], stat='variance',
                                       select_k=False, k=new_p,
                                       thresholding=True, variance_threshold=0,
                                       winsorize=False, winsorize_param=0.025,
                                       drop_outliers=False, drop_outliers_param=0.01,
                                       collinearity=False, collinearity_param=0.9)

# Screening features:
screening_cont.select_feat(df_train.drop(drop_vars, axis=1))

# Dataframe with variance by feature:
variation_num_feat = screening_cont.var_continuous_feat

# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

In [10]:
# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

print('updated_new_p: ' + str(screening_cont.k) + '.')
print('Number of selected features: ' + str(len(selected_feat)) + '.')
print('Number of features with zero variance: ' + str(len(zero_variance_feat)) + '.')

updated_new_p: 351.
Number of selected features: 1199.
Number of features with zero variance: 272.


In [11]:
# Variance by numerical variable:
print('\033[1mShape of variation_num_feat:\033[0m ' + str(variation_num_feat.shape) + '.')
variation_num_feat.head(10)

Shape of variation_num_feat: (1378, 3).


variance,feature,variation,missing
636,feat_637,1281.000000,0
591,feat_592,1281.000000,0
909,feat_910,1281.000000,0
983,feat_984,1281.000000,0
1123,feat_1124,1281.000000,0
1121,feat_1122,1281.000000,0
661,feat_662,1281.000000,0
1533,feat_1534,1273.454465,0
1067,feat_1068,1027.765432,0
1065,feat_1066,898.180556,0


<a id='winsorize'></a>

#### Winsorize treatment

In [12]:
# Declaring object for screening of feature:
screening_cont = VarScreeningNumerical(features = cont_vars, na_features = [], stat='variance',
                                       select_k=True, k=new_p,
                                       thresholding=False, variance_threshold=0,
                                       winsorize=True, winsorize_param=0.025,
                                       drop_outliers=False, drop_outliers_param=0.01,
                                       collinearity=False, collinearity_param=0.9)

# Screening features:
screening_cont.select_feat(df_train.drop(drop_vars, axis=1))

# Dataframe with variance by feature:
variation_num_feat = screening_cont.var_continuous_feat

# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

In [13]:
# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

print('updated_new_p: ' + str(screening_cont.k) + '.')
print('Number of selected features: ' + str(len(selected_feat)) + '.')
print('Number of features with zero variance: ' + str(len(zero_variance_feat)) + '.')

updated_new_p: 351.
Number of selected features: 351.
Number of features with zero variance: 272.


In [14]:
# Variance by numerical variable:
print('\033[1mShape of variation_num_feat:\033[0m ' + str(variation_num_feat.shape) + '.')
variation_num_feat.head(10)

Shape of variation_num_feat: (1378, 3).


variance,feature,variation,missing
636,feat_637,1281.000000,0
591,feat_592,1281.000000,0
909,feat_910,1281.000000,0
983,feat_984,1281.000000,0
1123,feat_1124,1281.000000,0
1121,feat_1122,1281.000000,0
661,feat_662,1281.000000,0
1533,feat_1534,1273.454465,0
1067,feat_1068,1027.765432,0
1065,feat_1066,898.180556,0


<a id='drop_outliers'></a>

#### Dropping outliers

In [15]:
# Declaring object for screening of feature:
screening_cont = VarScreeningNumerical(features = cont_vars, na_features = [], stat='variance',
                                       select_k=True, k=new_p,
                                       thresholding=False, variance_threshold=0,
                                       winsorize=False, winsorize_param=0.025,
                                       drop_outliers=True, drop_outliers_param=0.01,
                                       collinearity=False, collinearity_param=0.9)

# Screening features:
screening_cont.select_feat(df_train.drop(drop_vars, axis=1))

# Dataframe with variance by feature:
variation_num_feat = screening_cont.var_continuous_feat

# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

In [16]:
# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

print('updated_new_p: ' + str(screening_cont.k) + '.')
print('Number of selected features: ' + str(len(selected_feat)) + '.')
print('Number of features with zero variance: ' + str(len(zero_variance_feat)) + '.')

updated_new_p: 351.
Number of selected features: 351.
Number of features with zero variance: 272.


In [17]:
# Variance by numerical variable:
print('\033[1mShape of variation_num_feat:\033[0m ' + str(variation_num_feat.shape) + '.')
variation_num_feat.head(10)

Shape of variation_num_feat: (1378, 3).


variance,feature,variation,missing
636,feat_637,1281.000000,0
591,feat_592,1281.000000,0
909,feat_910,1281.000000,0
983,feat_984,1281.000000,0
1123,feat_1124,1281.000000,0
1121,feat_1122,1281.000000,0
661,feat_662,1281.000000,0
1533,feat_1534,1273.454465,0
1067,feat_1068,1027.765432,0
1065,feat_1066,898.180556,0


<a id='collinearity'></a>

#### Handling collinearity

In [18]:
# Declaring object for screening of feature:
screening_cont = VarScreeningNumerical(features = cont_vars, na_features = [], stat='variance',
                                       select_k=True, k=new_p,
                                       thresholding=False, variance_threshold=0,
                                       winsorize=False, winsorize_param=0.025,
                                       drop_outliers=False, drop_outliers_param=0.01,
                                       collinearity=True, collinearity_param=0.9)

# Screening features:
screening_cont.select_feat(df_train.drop(drop_vars, axis=1))

# Dataframe with variance by feature:
variation_num_feat = screening_cont.var_continuous_feat

# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

-------------------------------------------------------------------------
Number of selected features: 1
Candidate feature: feat_592
R2 from regression of candidate feature against selected features: 0.0
Candidate feature selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 2
Candidate feature: feat_910
R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 2
Candidate feature: feat_984
R2 from regression of candidate feature against selected features: 0.0
Candidate feature selected!
-------------------------------------------------------------------------


------------------------------------------------------------------------

R2 from regression of candidate feature against selected features: 0.8189
Candidate feature selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 20
Candidate feature: feat_1541
R2 from regression of candidate feature against selected features: 0.0612
Candidate feature selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 21
Candidate feature: feat_1622
R2 from regression of candidate feature against selected features: 0.0002
Candidate feature selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 22
Candidate feature: feat_1619
R2 from regression of candidate feature against selected feat

R2 from regression of candidate feature against selected features: 0.837
Candidate feature selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 40
Candidate feature: feat_176
R2 from regression of candidate feature against selected features: 0.7808
Candidate feature selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 41
Candidate feature: feat_1065
R2 from regression of candidate feature against selected features: 0.9316
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 41
Candidate feature: feat_658
R2 from regression of candidate feature against selected fea

R2 from regression of candidate feature against selected features: 0.9671
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 57
Candidate feature: feat_184
R2 from regression of candidate feature against selected features: 0.9357
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 57
Candidate feature: feat_985
R2 from regression of candidate feature against selected features: 0.8851
Candidate feature selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 58
Candidate feature: feat_188
R2 from regression of candidate feature against selected

R2 from regression of candidate feature against selected features: 0.9662
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 68
Candidate feature: feat_1069
R2 from regression of candidate feature against selected features: 0.8896
Candidate feature selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 69
Candidate feature: feat_792
R2 from regression of candidate feature against selected features: 0.9854
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 69
Candidate feature: feat_800
R2 from regression of candidate feature against selecte

...

R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 337
Candidate feature: feat_1203
R2 from regression of candidate feature against selected features: 0.8953
Candidate feature selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 338
Candidate feature: feat_1168
R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 338
Candidate feature: feat_1201
R2 from regression of candidate feature against selected

R2 from regression of candidate feature against selected features: 0.9156
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 344
Candidate feature: feat_1331
R2 from regression of candidate feature against selected features: 0.9279
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 344
Candidate feature: feat_94
R2 from regression of candidate feature against selected features: 0.9064
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 344
Candidate feature: feat_267
R2 from regression of candidate feature against s

R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 349
Candidate feature: feat_416
R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 349
Candidate feature: feat_437
R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 349
Candidate feature: feat_412
R2 from regression of candidate feature against selected f

R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 349
Candidate feature: feat_760
R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 349
Candidate feature: feat_761
R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 349
Candidate feature: feat_786
R2 from regression of candidate feature against selected f

R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 349
Candidate feature: feat_400
R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 349
Candidate feature: feat_405
R2 from regression of candidate feature against selected features: 1.0
Candidate feature not selected!
-------------------------------------------------------------------------


-------------------------------------------------------------------------
Number of selected features: 349
Candidate feature: feat_404
R2 from regression of candidate feature against selected f

In [19]:
# List of features with the highest variance:
selected_feat = screening_cont.selected_feat

# List of features with no variance:
zero_variance_feat = screening_cont.no_var_continuous_feat

print('updated_new_p: ' + str(screening_cont.k) + '.')
print('Number of selected features: ' + str(len(selected_feat)) + '.')
print('Number of features with zero variance: ' + str(len(zero_variance_feat)) + '.')

updated_new_p: 351.
Number of selected features: 351.
Number of features with zero variance: 272.


In [20]:
# Variance by numerical variable:
print('\033[1mShape of variation_num_feat:\033[0m ' + str(variation_num_feat.shape) + '.')
variation_num_feat.head(10)

Shape of variation_num_feat: (1378, 3).


variance,feature,variation,missing
636,feat_637,1281.000000,0
591,feat_592,1281.000000,0
909,feat_910,1281.000000,0
983,feat_984,1281.000000,0
1123,feat_1124,1281.000000,0
1121,feat_1122,1281.000000,0
661,feat_662,1281.000000,0
1533,feat_1534,1273.454465,0
1067,feat_1068,1027.765432,0
1065,feat_1066,898.180556,0


<a id='corr_based_selection'></a>

### Features selection based on correlation

In [21]:
cont_vars = [c for c in df_train.columns if c not in drop_vars]

# Declaring object for screening of feature:
screening_cont = CorrScreeningNumerical(features = cont_vars, na_features = [], stat='variance',
                                        corr_threshold=0.8,
                                        winsorize=False, winsorize_param=0.025,
                                        drop_outliers=False, drop_outliers_param=0.01)

# Screening features:
screening_cont.select_feat(df_train.drop(drop_vars, axis=1))

# Dataframe with variance by feature:
variation_num_feat = screening_cont.var_continuous_feat

print('Number of selected features: ' + str(len(screening_cont.selected_feat)) + '.')
print('\033[1mShape of variation_num_feat:\033[0m ' + str(variation_num_feat.shape) + '.')
variation_num_feat.head(10)

/home/matheus_rosso/Arquivo/Materiais/Codes/high_dimensionality/Codes/screening_features.py:95: RuntimeWarning: Degrees of freedom <= 0 for slice.
  'variation': [np.nanvar(data[f]) for f in self.features]})


Number of selected features: 516.
Shape of variation_num_feat: (1378, 3).


variance,feature,variation,missing
636,feat_637,1281.000000,0
591,feat_592,1281.000000,0
909,feat_910,1281.000000,0
983,feat_984,1281.000000,0
1123,feat_1124,1281.000000,0
1121,feat_1122,1281.000000,0
661,feat_662,1281.000000,0
1533,feat_1534,1273.454465,0
1067,feat_1068,1027.765432,0
1065,feat_1066,898.180556,0
